# State_dependency (With langchain and memory)

In [1]:
# Unconmenti this line and replace 'key.txt' with your key
import os
os.environ["OPENAI_API_KEY"] = open("key.txt").read().strip() # run this code before engaging LLM model
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor # the tools needed to create a sort of chain with directives

# ============================
# 1) creazione tool 'settings'
# ============================

# print('fist page') The model only call the tools and not the whole page everytime

def set_system_settings(check_Key_Status:str,update_Key_Status:bool|str=None):
    """Check wifi and battery status. (Ricorda che il modello impara dai 'return' di queste tool e prende decisioni di conseguenza)""" 

    check_Key_Status = check_Key_Status.lower() # because the model can create uppercase parameters
    print(f"check_Key_Status here = '{check_Key_Status}' ",)
   
    getSettingsData = {
      "metadata": {
        "flow": {
           "initial_databases": {
                "SETTING": [
                    {
                        "device_id": "04640c30-ec89-5a3c-9fb4-4af3101c8e04",
                        "place_id": "ChIJ_Yjh6Za1j4AR8IgGUZGDDTs",
                        "latitude": 37.33464379999999,
                        "longitude": -122.008972,
                        "formatted_address": "One Apple Park Way, Cupertino, CA 95014, USA",
                        "wifi": False, # update_Key_Status if 'wifi' in check_Key_Status and update_Key_Status else False, # conditional inline expression
                        "location_service": False,
                        "cellular": False,
                        "low_battery_mode": True, #update_Key_Status if 'low_battery_mode' in check_Key_Status and update_Key_Status else True,
                        "utc_offset_seconds": -25200,
                        "locale": "en_US"
                  }
                ]
              }
            }
          }
        }
    
    # get the value of the args genereted by the model
    key_value = getSettingsData['metadata']['flow']['initial_databases']['SETTING'][0][check_Key_Status] # get the value of a system key create as argument from the model as we toldit in the systemMessage

    # return F" {check_Key_Status} status = {data_value}"
    if check_Key_Status and update_Key_Status is None: # Just checking the keyValue
       print(f"before update value = {check_Key_Status}:{key_value}") 
  
       return f" Only check_Key_Status: {check_Key_Status} status = {key_value}" # deve essere oltre lo if se vuoi che questo return avviene entro queste condizioni
    elif check_Key_Status and isinstance(update_Key_Status,(bool,str,int)) : # update the keyValue with true false value. update_Key_Status kan be bool, str or int
        
        print(f"after update value = {check_Key_Status}:{update_Key_Status}") 
         # Same updatings in memory. We assume that check_Key_Status muste be wifi or lower_battery_mode, so we can update it correctly in memory.
         #  We can in future have a Parameters that duble check that, but for now its good this way
        memory.set_state_action('Key_Status',f'{check_Key_Status}', update_Key_Status)

        return f"assistant_summary: {check_Key_Status} has been updated to {update_Key_Status}. Current device state: {memory._state_actions['Key_Status']}" # Così il modello impara subito anche il nuovo stato, utile per evitare richiami ridondanti al tool
        # il modello impara da questi returns, per completare la task.
    else:
        return f" please enter valid arguments" 


    
def search_place():
    """Imitating a search tools that invoke goole maps. 
    (Ricorda che grazie a agent_scratchpad che utilizza il "think step by step method", il modello impara dai return di queste tool e prende decisioni di conseguenza)"""


    settings_values = memory.get_state_action('Key_Status') # Get the current value of wifi and low_battery_mode settings stored in llm memory

    if settings_values and settings_values['wifi'] is True and settings_values['low_battery_mode'] is False:

         print(f"search_place() wifi_status: {settings_values['wifi'] }, search_place() low_battery_mode: {settings_values['low_battery_mode']}")

         getPlaceData = [
          [
      {
      "formatted_address": "1600 Amphitheatre Pkwy, Mountain View, CA 94043",
      "geometry": {
        "location": {
          "lat": 37.4220541,
          "lng": -122.0853242
        }
      },
      "name": "Googleplex",
      "place_id": "ChIJj61dQgK6j4AR4GeTYWZsKWw",
      "rating": 4.2,
      "types": [
        "point_of_interest",
        "establishment"
        ]
       }
      ]
     ]
         return getPlaceData[0][0]['formatted_address']
    else:
        
         return 'Enable wifi and disable low_battery_status to get the searched Adress' # This return is important for the model to know correctly
         # what to do. Funge da un ulteriore direttiva, come se l avessimo scritta in systemMessage. Senza questo return, il modello non capisce correttamente che
         # deve fare l update di wifi e low_battery_mode (anche se lo avevamo specificato nel tool set_system_setting()). Quindi il modello capisce anche 
         # dal return dei tools (e non solo dalle direttive e tool descriptions)
         
        # return 'wifi has to be on, and low_battery_status has to be off to get the searched Adress' # This works fine, but we defined it in the 
        # set_system_settings() tool 'description' so the model already knows about in and it doenst have to find it at runtime (like when it get that return
        # message when ivoking this function the first time it)


# =======================================
# 2) define the Tools structure
# =======================================

from langchain.tools import StructuredTool

SETTING = [# to get the keys value to insert in the description
  { # endta dentro il liste e prendi le key dell obj
    "device_id": "04640c30-ec89-5a3c-9fb4-4af3101c8e04",
    "place_id": "ChIJ_Yjh6Za1j4AR8IgGUZGDDTs",
    "latitude": 37.33464379999999,
    "longitude": -122.008972,
    "formatted_address": "One Apple Park Way, Cupertino, CA 95014, USA",
    "wifi": False,
    "location_service": False,
    "cellular": False,
    "low_battery_mode": True,
    "utc_offset_seconds": -25200,
    "locale": "en_US"
    }
    ]
# print(SETTING)

Settin_keys = SETTING[0].keys()  # entra dentro il list e prendi le key dell obj
Settin_keys = list(Settin_keys) # fai diventare queste key da obj a list

    
settings_tool = StructuredTool.from_function(
    func=set_system_settings,
    name='set_system_settings',
    description="Checks or updates system settings (wifi, low_battery_mode)." # you can list all the keyword if needed, like the description below
    # description=f"""Check the settings systems of a device. Requires check_Key_Status(use this keys on the list:{Settin_keys} to get a key state value), update_Key_Status(if needed, to update the 'check_Key_Status' state)"""
    # BOTH descriptions works. The commented one is optimal when you need to check more key_state, without writing them one by one 
)

search_tool = StructuredTool.from_function(
    func=search_place,
    name='search_place',
    description="Searches for a place; Only returns valid data when wifi is on and low_battery_mode is off." # Con questa description, il modello verifica direttamente il setting tool prima di invocare il search_tool
    # description='A search tool that helps find places or locations' # Con questa description invece il modello solitamente invoca search_tool al primo turn, e di conseguenza aggiusta i settings dove occorre.
)

tools = [settings_tool, search_tool]

tool_descriptions = "\n".join(
    [f"- {t.name}({', '.join(t.args_schema.__annotations__.keys())}) -> {t.description}"
     for t in tools]
) # describe the tools, we add it in state_dependency_system as a source of tools data

# print('tool_descriptions here =',tool_descriptions)

# =======================================
# 3) Creazione SystemMessage
# ======================================= 

state_dependency_system = f"""
You are a conversational assistant embedded in a mobile device. The user is likely interacting visually with the screen. Avoid excessive explanations and focus on highlighting only key information. Be factual, clear, and efficient. The user recently returned from international travel, is sleep-deprived, and prefers late-day activities. They have a strong preference for Asian food and often refer to their yoga instructor Lara as “my guide.” When suggesting options or actions, incorporate these preferences naturally without explicit reminders.
Maintain a confident, friendly, and respectful tone. Avoid formal or overly scripted responses. The assistant should behave like a competent personal secretary: efficient, light-toned, and responsive, capable of inferring context when it is clear but cautious where ambiguity exists. Never use pleasantries like “please” or “thank you,” and avoid robotic phrasing.

1) - Do not assume tool parameters unless provided by the user, the prompt history, or the system context.

2) - Understand the user intends and call the rigth tools.

3) - BEFORE calling any tool: produce a short HUMAN-FACING plan sentence prefixed with "assistant_plan:" describing what you will do next (example: "assistant_plan: i'll check the device settings and enable Wi-Fi if needed, then search for a location").

4) - AFTER tool outputs or after state changes, produce a clear HUMAN-FACING summary prefixed with "assistant_summary:" describing the result and next step (example: "assistant_summary: Wi-Fi was off → I turned it on. Now I'll retry the search.").

5) - Use ISO 8601 for any timestamp if needed.

6) - NEVER autput chain-of-thought. The plan and summary are the only "thinking" visible to the user.

7) - Whenever a user ask a question, Respond conversionally but always follow rules nr 3 and 4 around tool usage.

8) - Whenever the user asks about a place or location, ALWAYS re-run the `search_place` tool to ensure up-to-date data, even if a previous result exists in memory.

9) - Check the stored system settings in memory before making any decision. Only call set_system_settings if the desired setting is not already correct.

10) - When the user asks about a place, location, or address (e.g., 'where is', 'find', 'locate', 'search for', etc.), you must always call the `search_place` tool, regardless of memory state or previous results.

11) - Never assume a previous search result is still valid; every new user query must trigger a new search tool call.


You have access to the following tools:
{tool_descriptions}

When the user's request clearly matches one of these actions, 
call the appropriate tool by reasoning about its name and description..
"""
# When the user gives an instruction, decide if it’s:
# - A general chat → reply with advice, no tool use.

# IMPORTANTI DIRETTIVE
# senza questa istruzione, il modello e propenso a dare nomi degli args errati (tipa low_battery_status invece di low_battery_mode)
# - search_place tool returns valid search data only if 'wifi' is 'on' and low_battery_mode is 'off'. 

# 1) - Do not assume tool parameters unless provided by the user, the prompt history, or the system context. Questa direttiva ci serve perche le args non le deve decidere il modello, ma sono quelle, a differenza della task Error_recovery dove il modello deve scegliere
# se inserire altre args per trovare un evento(e.g. 'add date upperbound'(broadening)) oppure ometterle (e.g. omit date lowerbound (narrowing))

# 8) - Whenever the user asks about a place or location, always re-run the `search_place` tool to ensure up-to-date data, even if a previous result exists in memory.
# 10) - When the user asks about a place, location, or address (e.g., 'where is', 'find', 'locate', 'search for', etc.), you must always call the `search_place` tool, regardless of memory state or previous results.
# 11) - Never assume a previous search result is still valid; every new user query must trigger a new search tool call.
# Questa direttive sono importanti perche se per esempio chiediamo di nuovo 'Where is googleplex?' il modello risponde con i dati in memoria della stessa domanda
# precedente. Mentre se usiamo questa direttiva, forziamo il modello a chiamare sempre la funzione anche se abbiamo fatto la stessa domanda identica
# Questo serve ad avere output di dati dal tool e non dalla memoria(molto importante)

# 9) - Check the stored system settings in memory before making any decision. Only call set_system_settings if the desired setting is not already correct.
# Questa direttiva e' molto importante perche avendo i dati salvati in memoria non andremo a chiamare inutilmente una tool se non e' necessario (come descritto nella guideline)

# =======================================
# 3) Creazione del Prompt Template
# =======================================

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(state_dependency_system), # The systeMessage
    MessagesPlaceholder(variable_name="chat_history"), # chat_history (niettare contenuti dinamici nel prompt.)
    MessagesPlaceholder(variable_name="system_context"), # system_context (niettare contenuti dinamici nel prompt.)
    HumanMessagePromptTemplate.from_template("{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # 'agent_scratchpad' È il meccanismo chiave per il reasoning step-by-step del modello.
    # Contiene i risultati intermedi (tool call + return) della sessione corrente, cioè:
    # Grazie ad esso il modello apprende anche dal 'return' delle tool invocate, e non solo dalle systemMessage.
    # Per esempio, in questo caso quando invochiamo la search tool, il return informa il modello che "wifi deve essere on e lower_setting_state deve essere off"
    # e grazie a questo return informativo(che ricordiamo non deve essere troppo lungo), l'agent_scratchpad aiuta il modello a capire di agiornare i settings prima di invocare 
    # nuovamente il 'search tool'. 
    # In parole semplici: 
    # "il modello “vede” che la causa dell’errore è il Wi-Fi spento e può decidere da solo di chiamare set_system_settings() per sistemarlo.""
])


from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseMessage, HumanMessage



# 4 Creazione della classe SafeConversationMemory(ConversationBufferMemory)

class SafeConversationMemory(ConversationBufferMemory): # questa classe viene invocata ogni volta l utente interagisce con il modello, ma non viene ricreata da zero
    # viene riutilizzata come istanza persistente, e al suo interno la chat history e gli stati vengono aggiornati.
    # ogni volta che comunichiamo con il modello: agent_executor.invoke({"input": "accendi il Wi-Fi"}), LangChain fa questo dietro le quinte:

    # 1) Chiama memory.load_memory_variables() (viene chiamato prima che l'agente costruice il prompt. 
    # Serve per elaborare i dati in memoria e servirli al modello prima che esso da' una risposta)
    
    # Serve per:
    
    # leggere la chat history precedente,
    
    # leggere eventuali stati (_state_actions),
    
    # costruire un contesto strutturato in chiave BaseMessage tipo:
#     {
#   "chat_history": [HumanMessage(...), AIMessage(...)],
#   "system_context": [HumanMessage(content="{'wifi': False, 'low_battery_mode': True}")]
#    } 👉 Questo viene poi iniettato nel prompt grazie a MessagesPlaceholder (insieme al system message e all’input utente).

# 2) L’LLM genera una risposta (eventualmente invocando tool)

# 3) dopo ogni risposta del modello viene chiamata la funzione 'memory.save_context' che non vediamo qua ma avviene di sottobanco e appartiene ConversationBufferMemory.
     # Questa funzione 'aggiorna' il HumanMessage, AIMessage e system_context, in modo che alla prossima domanda fatta al modello, questo possa 
     # prendere esempio dalle domande, risposte e system_context data del turno precedente (per esempio dopo il primo turn, memory.save_context salva gli stati aggiornati di
     # wifi e low_battery_mode ). E' al prossimo turn 'memory.save_context' aggiorna memory.load_memory_variables() e poi il processo si ripete
     # dal punto 1) quando lo user pone un altra domanda al modello

# Entrambi i metodi che hai citato (save_context() e load_memory_variables()) appartengono alla classe base BaseMemory di LangChain, 
# e sono implementati concretamente nella sottoclasse ConversationBufferMemory (e quindi anche nella tua SafeConversationMemory)

# 'load_memory_variables' viene chiamato prima che l'agente costruice il prompt (cosi prepariamo il modello a tenere conto dei dati della memory prima di leggere promp). 
# 'save_context' viene chiamato dopo che il modello ha risposto(cosi aggiorniamo per il prossimo turn, e 'load_memory_variables' prepara il modello con i nuovi dati aggiornati da 'memory.save_context' dopo che il modello ha risposto)
    
    """
    questa classe estende ConversationBufferMemory, con lo scopo di renderla piu' “intelligente” sicura.
    Enhanced memory that:
    - Cleans malformed message content (list/dict -> string)
    - Keeps track of system_settings status
    - Add the actual status into the model through context by adding it in memory, so in the systemMessage you tell the model to verify in memory to make clever decisions 
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        print('Hit self, **kwargs')
       
        self._state_actions = {'Key_Status':{ # default status. Per far capire al modello che deve updatare questi date per accedere a serch tool
            'wifi':False,
            'low_battery_mode':True
        }} # This ll be updated when we model calls set_state_action, (we can see updates when invoking memory data).
        # Serve a memorizzare lo stato reale del sistema (non solo le parole nella chat).


    # ============================================================
    # 🧠 1️⃣ Methods to manage pending actions
    # ============================================================

    def set_state_action(self, name: str, key:str, value:str|bool):# ( data: self, name: str, data:dict) # we call this def from the tool def create_reservation(name,data)
        """Store a pending action waiting for user confirmation."""
        # self._state_actions[name] = data
        self._state_actions[name][key] = value # queste value e key saranno sempre le stesse 2('wifi' e 'low_batteri_mode') e per questo non si sommeranno, ma cambieranno solo le value (true or false) 
        # Vengono aggiornate uno alla volta, in ordine del modello quando invoka l'update delle key value nella tool set_system_settings

    def get_state_action(self, name: str): # controls if there are any pending actions when the model calls add_to_calender(name) Tool
        """Retrieve a specific pending action."""
        return self._state_actions.get(name) # invocando questa funzione avremo l attuale status delle key 'wifi' e 'low_batteri_mode'

    # ============================================================
    # 🧹 2️⃣ Override: cleaning + injecting context for the model
    # ============================================================

    def load_memory_variables(self, inputs):
        """Return cleaned chat history + pending actions as context."""
        raw = super().load_memory_variables(inputs) # il nome del method della classe ConversationBufferMemory, che qui estendiamo e modifichiamo per fare 
        # pulizie e aggiungere system_context in formato BaseMessage. (deve avere questo nome load_memory_variables)

        print('hit load_memory_variables and inputs :',inputs) # {'input': 'where is googleplex?'}
        # ---- Clean malformed messages (lists/dicts converted to strings)
        if "chat_history" in raw:  # chat_history e' la key che abbiamo dato al nostro memoryKey quando abbiamo definito 'memory' e nel prompt
            cleaned = []
            print('hit chat_history')
            for msg in raw["chat_history"]:
                if isinstance(msg, BaseMessage): # BaseMessage(HumanMessage(content:'user_input') and AIMessage(content:'ai response')) is the rigth type for our model. Its the type the model require to work without errors.
                    # It is like that:
                    # {'chat_history': [HumanMessage(content='I need a reservation', additional_kwargs={}, response_metadata={}), AIMessage(content='Could you please clarify what type of reservation you need? For example, is it for a restaurant, hotel, or something else? Additionally, please provide any specific details like the date, time, and number of people if applicable.', additional_kwargs={}, response_metadata={})}
                    # so basically our memory stored in ConversationBufferMemory is stored in this way showed in HumanMessage and AIMessage
                    msg.content = str(msg.content) # turn (list, dict) content in str type to not throw errors
                    cleaned.append(msg)
                elif isinstance(msg, str): # if there are some 'fully string type' value that can cause errors(like not structured baseMessage data), instead of BaseMessage structure in our
                      # ConversationBufferMemory chat_history , then turn this string  to HumanMessage(). This 'string' bug can hapend when the model calls a tool, creating a 
                      # 'string with the name of the tool invoked', that the Model doesnt support and cannot parse.
                      #(HumanMessage is part of baseMessage for related to the user_input. there is the exemple right up in 'if isinstance(msg, BaseMessage)'). 
                     cleaned.append(HumanMessage(content=msg)) 
            raw["chat_history"] = cleaned # puliamo l object chat_history
            print('check raw ->', raw)

        # ---- Add pending actions as extra conversational context. Questo previene chiamate al tool "reservation" quando invokiamo il tool add_to_calender
        # inseriamo in memoria che c'e un pending tool ogni volta che il tool "reservation" viene invocato, e nel systemMessage diciamo specificamente
        # al modello di non invocare nuovamente il tool "reservation" quando decidiamo di mettere questo "reservatio event" come evento nel tool "calender"
        # infine ogni volta che il modello inserisce l'evento sul tool calender, il pendig state ritorna vuoto, cosicche' il processo si ripete e 
        # non ci ripeta lo stesso problema
        if self._state_actions: # if _state_actions not empty, cioe' se ci sono dei pending_state 
            context_str = (
                f"(State actions currently stored: {self._state_actions})" # questo va a' finire nella memory nel formato humaMessage come bediamo sotto
            )
            print('check state_actions ->', context_str)
        else:
            context_str = "(No state actions stored.)"
        
        # aggiungiamo pending_actions in formato HumanMessage
        raw["system_context"] = [HumanMessage(content=context_str)] # aggiungiamo un messaggio extra che contiene i dati di ogni turn delle values dei settngs
        # sotto il prefisso 'system_context'(inventato da me) che poi inseriamo nel prompt/AIMessagePlaceHolder, cosi ad ogni turno il modello tiene 
        # conto dei dati ogiornati o meno di settings del turno precedente
        # Lo stesso vale per chat_history
        # trasformiamo il pending action in uno HumanMessage in modo che il formato sia
        # corretto da inserire nella memory e da essere compatimile con il formato del modello.
        # Ora  Il modello “vede” il pending_action nel contesto e può comportarsi di conseguenza (nei punti 5 e 6 del systemMessage diciamo al
        # di guardare i pending_state della memory e cosa fare se ce ne sono di attivi. 
        
        return raw  # ritorniamo la struttura di ConversationBufferMemory() piu' pulita e' con l aggiunt di pending_actions


memory = SafeConversationMemory( # istanza della classe SafeConversationMemory. Qui passiamo anche questi parametri definiti qua dentro alla classe
    memory_key="chat_history", # crea il BaseMessage della cronologia tra user e model(HumanMessage,AiMessage)
    return_messages=True,
    input_key="input"  # 👈 forza un solo input
)


# =======================================
# ✅  Creazione modello
# =======================================

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.4)


# =======================================
# ✅  Creazione agente
# =======================================

agent = create_openai_functions_agent( # creiamo il nostro agente strutturato
    llm=llm,
    tools = tools,
    prompt=prompt
)


agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools= tools, 
    memory=memory,
    verbose=True,
    handle_parsing_errors=True, 
)


Hit self, **kwargs


In [2]:
user_input = "where is googleplex?" 
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers


hit load_memory_variables and inputs : {'input': 'where is googleplex?'}
hit chat_history
check raw -> {'chat_history': []}
check state_actions -> (State actions currently stored: {'Key_Status': {'wifi': False, 'low_battery_mode': True}})


> Entering new AgentExecutor chain...

Invoking: `set_system_settings` with `{'check_Key_Status': 'wifi', 'update_Key_Status': True}`


check_Key_Status here = 'wifi' 
after update value = wifi:True
assistant_summary: wifi has been updated to True. Current device state: {'wifi': True, 'low_battery_mode': True}
Invoking: `search_place` with `{}`


Enable wifi and disable low_battery_status to get the searched Adress
Invoking: `set_system_settings` with `{'check_Key_Status': 'low_battery_mode', 'update_Key_Status': False}`


check_Key_Status here = 'low_battery_mode' 
after update value = low_battery_mode:False
assistant_summary: low_battery_mode has been updated to False. Current device state: {'wifi': True, 'low_battery_mode': False}
Invoking: `sear

In [ ]:
# No useless function call when it is a general question or a simple 'tnx'
user_input = "tnx chat" #"Aggiungi una sessione di stretching alle 7:30."
response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
print(response["output"]) # output where you get the answers

hit load_memory_variables and inputs : {'input': 'tnx chat'}
hit chat_history
check raw -> {'chat_history': [HumanMessage(content='where is googleplex?', additional_kwargs={}, response_metadata={}), AIMessage(content='assistant_summary: The Googleplex is located at 1600 Amphitheatre Pkwy, Mountain View, CA 94043.', additional_kwargs={}, response_metadata={})]}
check state_actions -> (State actions currently stored: {'Key_Status': {'wifi': True, 'low_battery_mode': False}})


> Entering new AgentExecutor chain...
assistant_summary: No action needed. If you need anything else, just let me know!

> Finished chain.
assistant_summary: No action needed. If you need anything else, just let me know!


In [ ]:
# USE THOSE INPUT TO SEE HOW THE MODEL REACT

# user_input = "Ok, where is Googleplex?" # the model always invoke search_place tool to guarantee up-to-date data
# response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
# print(response["output"]) # output where you get the answers

In [ ]:
# secondo turn: il modello perche i parametri wifi e lower_battery_status sono salvati corretti in in memoria e quindi decide che non serve 
# richiamare la search_place(), e risponde “Il Googleplex si trova a Mountain View…” basandosi solo sulla memoria.

# USE THOSE INPUT TO SEE HOW THE MODEL REACT
# user_input = "where is California?" 
# response = agent_executor.invoke({"input": user_input}) # input is where you add the question, 
# print(response["output"]) # output where you get the answers
